In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

In [ ]:
train_data_file = "../data/zhengqi_train.txt"
test_data_file = "../data/zhengqi_test.txt"

train_data = pd.read_csv(train_data_file, sep='\t', encoding='utf-8')
test_data = pd.read_csv(test_data_file, sep='\t', encoding='utf-8')

In [ ]:
# 查看数据的基本信息，是否有缺失值，数据类型等等
train_data.info()
test_data.info()

In [ ]:
# 查看统计后数据的数目，均值，最大最小值
train_data.describe()
test_data.describe()

In [ ]:
train_data.head()
test_data.head()

In [ ]:
# fig = plt.figure(figsize=(4,6))
# sns.boxplot(train_data['V0'], orient="v",width=0.5)
# 箱型图，上四分线，下四分线
#列表头
column = train_data.columns.tolist()[:39]
fig = plt.figure(figsize=(80,60),dpi=75)
for i in range(38):
    plt.subplot(7,8,i+1)
    sns.boxplot(train_data[column[i]],orient="v",width=0.5)
    plt.ylabel(column[i],fontsize=36)

In [ ]:
# 所有变量的直方图 以及 Q-Q图（通过比较数据和正态分布的分位数是否相等）
plt.figure(figsize=(10,5))
ax = plt.subplot(1,2,1)
sns.distplot(train_data['V0'],fit=stats.norm)
ax = plt.subplot(1,2,2)
res = stats.probplot(train_data['V0'],plot = plt)


In [ ]:
train_cols = 6
train_rows = len(train_data.columns)
plt.figure(figsize=(4*train_cols,6*train_rows))
i = 0
for col in train_data.columns:
    i += 1
    ax = plt.subplot(train_rows,train_cols,i)
    sns.distplot(train_data[col],fit=stats.norm)
    
    i += 1
    ax = plt.subplot(train_rows,train_cols,i)
    res = stats.probplot(train_data[col],plot=plt)

In [ ]:
# KDE分布图,核密度估计图，查看训练集和测试集的分布情况
plt.figure(figsize=(8,4),dpi=150)
ax = sns.kdeplot(train_data['V0'],color="Red",shade=True)
ax = sns.kdeplot(test_data['V0'],color="Blue",shade=True)
ax.set_xlabel('V0')
ax.set_ylabel("Frequency")
ax = ax.legend(["train","test"])

In [ ]:
dist_cols = 6
dist_rows = len(test_data.columns)
plt.figure(figsize=(4*dist_cols,4*dist_rows))
i = 1
for col in test_data.columns:
    ax = plt.subplot(dist_rows,dist_cols,i)
    ax = sns.kdeplot(train_data[col],color="Red",shade=True)
    ax = sns.kdeplot(test_data[col],color="Green",shade=True)
    ax.set_xlabel(col)
    ax.set_ylabel("Frequency")
    ax = ax.legend(["Train","Test"])
    i += 1


In [ ]:
# 线性回归关系图，分析变量间的线性关系
fcols = 2
frows = 1
plt.figure(figsize=(8,4),dpi=150)
ax = plt.subplot(1,2,1)
sns.regplot(x='V0',y='target',data=train_data,ax=ax,
            scatter_kws={'marker':'.','s':3,'alpha':0.3},line_kws={'color':'k'})
plt.xlabel('V0')
plt.ylabel('Target')

ax = plt.subplot(1,2,2)
sns.distplot(train_data['V0'].dropna())
plt.xlabel('V0')
plt.show()

In [ ]:
fcols = 6
frows = len(test_data.columns)
plt.figure(figsize=(5*fcols,4*frows))

i = 0
for col in test_data.columns:
    i += 1
    ax = plt.subplot(frows,fcols,i)
    sns.regplot(x=col,y='target',data=train_data,ax=ax,
    scatter={'marker':'.','s':3,'alpha':0.3},
    line_kws={'color':'k'})
    plt.xlabel(col)
    plt.ylabel('target')
    i += 1
    ax = plt.subplot(frows,fcols,i)
    sns.distplot(train_data[col].dropna())
    plt.xlabel(col)

In [ ]:
# 查看特征变量的相关性;主要用于判断线性相关
# 0-0.09无关;0.1-0.3弱相关;0.3-0.5中等相关；0.5-1强相关
pd.set_option('display.max_columns',10)
pd.set_option('display.max_rows',10)
data_train1 = train_data.drop(['V5','V9','V11','V17','V22','V28'],axis=1)
train_corr = data_train1.corr()
# print(train_corr)
ax = plt.subplots(figsize=(20,16))
ax = sns.heatmap(train_corr,vmax=.8,square=True,annot=True)
annot = True

In [ ]:
k = 10
cols = train_corr.nlargest(k,'target')['target'].index
# cm = np.corrcoef(train_data[cols].values.T)
# print(cm)
hm = plt.subplots(figsize=(10,10))
hm = sns.heatmap(train_data[cols].corr(),annot=True,square=True)
# plt.show

In [ ]:
threshold = 0.5
corrmat = train_data.corr()
top_corr_features = corrmat.index[abs(corrmat["target"])>threshold] 
# print(top_corr_features)
plt.figure(figsize=(10,10))
g = sns.heatmap(train_data[top_corr_features].corr(),annot =True,cmap="RdYlGn")

In [ ]:
# 相关系数矩阵的运算
corr_matrix = data_train1.corr().abs()
drop_col = corr_matrix[corr_matrix["target"] < threshold].index
# data_all.drop(drop_col,axis = 1,inplace =True)

In [ ]:
# box-cox变换，1：数据归一化处理
drop_columns = ['V5','V9','V11','V17','V22','V28']
train_x = train_data.drop(['target'],axis=1)
# data_all = pd.concat([train_data,test_data],axis=0,ignore_index=True)
data_all = pd.concat([train_data,test_data])
data_all.drop(drop_columns,axis=1,inplace=True)
data_all.head()
cols_numeric = list(data_all.columns)

def scale_minmax(col):
    return (col-col.min())/(col.max()-col.min())

data_all[cols_numeric]=data_all[cols_numeric].apply(scale_minmax,axis=0)
data_all[cols_numeric].describe()

In [ ]:
# 特征工程
# 异常值分析;箱型图
plt.figure(figsize=(18,10))
plt.boxplot(x=train_data.values,labels=train_data.columns)
plt.hlines([-7.5,7.5],0,40,colors='r')
plt.show()

In [ ]:
# 剔除异常值
train_data = train_data[train_data['V9']>-7.5]
test_data = test_data[test_data['V9']>-7.5]
display(train_data.describe())
display(test_data.describe())

In [ ]:
# 归一化处理
from sklearn import preprocessing
features_columns = [col for col in train_data.columns if col not in ['target']]
#用已有的训练数据，构建转换器
min_max_scaler = preprocessing.MinMaxScaler()
min_max_scaler = min_max_scaler.fit(train_data[features_columns])

train_data_scaler = min_max_scaler.transform(train_data[features_columns])
test_data_scaler = min_max_scaler.transform(test_data[features_columns])

train_data_scaler = pd.DataFrame(train_data_scaler)
train_data_scaler.columns = features_columns

test_data_scaler = pd.DataFrame(test_data_scaler)
test_data_scaler.columns = features_columns
train_data_scaler['target'] = train_data['target']

display(train_data_scaler.describe())
display(test_data_scaler.describe())

In [ ]:
drop_col = 6
drop_row = 1
plt.figure(figsize=(5*drop_col,5*drop_row))

for i,col in enumerate(["V5","V9","V11","V22","V28"]):
    ax = plt.subplot(drop_row,drop_col,i+1)
    ax = sns.kdeplot(train_data[col],color='Red',shade=True)
    ax = sns.kdeplot(test_data_scaler[col],color='Blue',shade=True)
    ax.set_xlabel(col)
    ax.set_ylabel('Frequency')
    ax = ax.legend(["train","test"])
plt.show()

In [ ]:
# 去除多重共线性分析
# 多重共线性方差膨胀因子
from statsmodels.stats.outliers_influence import variance_inflation_factor
new_numerical =['V0','V2','V3','V4','V5','V6','V10','V11','V13','V15','V16','V18',
                'V19','V20','V22','V24','V30','V31','V37']
X = np.matrix(train_data_scaler[new_numerical])
VIF_list = [variance_inflation_factor(X,i) for i in range(X.shape[1])]
VIF_list

In [ ]:
# 主成分分析，去除多重共线性
from sklearn.decomposition import PCA
pca = PCA(n_components=0.9)
new_train_pca_90 = pca.fit_transform(train_data_scaler.iloc[:,0:-1])
new_test_pca_90 = pca.transform(test_data_scaler)
new_train_pca_90 = pd.DataFrame(new_train_pca_90)
new_test_pca_90 = pd.DataFrame(new_test_pca_90)
new_train_pca_90['target'] = train_data_scaler['target']
new_train_pca_90.describe()
# train_data_scaler.describe()


In [ ]:
pca = PCA(n_components=16)
new_train_pca_16 = pca.fit_transform(train_data_scaler.iloc[:,0:-1])
new_test_pca_16 = pca.transform(test_data_scaler)
new_train_pca_16 = pd.DataFrame(new_train_pca_16)
new_test_pca_16 = pd.DataFrame(new_test_pca_16)
new_train_pca_16['target'] = train_data_scaler['target']
new_train_pca_16.describe()

In [ ]:
#线性回归  
from sklearn.linear_model import LinearRegression
#K近邻回归
from sklearn.neighbors import KNeighborsRegressor
#决策树回归 
from sklearn.tree import DecisionTreeRegressor
# 随机森林回归
from sklearn.ensemble import RandomForestRegressor
# 支持向量回归
from sklearn.svm import SVR
import lightgbm as lgb

# 切分数据和评价指标
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit

In [ ]:
# 采用PCA的16维度数据
new_train_pca_16 = new_train_pca_16.fillna(0)
# print(new_train_pca_16)
train = new_train_pca_16[new_test_pca_16.columns]
target = new_train_pca_16['target']
# 切分数据
train_data,test_data,train_target,test_target = train_test_split(train,target,test_size=0.2,random_state=0)

In [ ]:
# 多元线性回归
# Q：精度低，可能特征存在共线问题；
# 要先归一化处理
clf = LinearRegression()
clf.fit(train_data,train_target)
score = mean_squared_error(test_target,clf.predict(test_data))
print("多元线性回归：",score)


In [ ]:
# K近邻回归
# Q：计算量大，要调参（适用于小的数据集）
clf = KNeighborsRegressor(n_neighbors=8)
clf.fit(train_data,train_target)
score = mean_squared_error(test_target,clf.predict(test_data))
print("k近邻居：",score)

In [ ]:
# 随机森林
# Q：结果不易解释，但精度高
clf = RandomForestRegressor(n_estimators=200)
clf.fit(train_data,train_target)
score = mean_squared_error(test_target,clf.predict(test_data))
print("随机森林",score)

In [ ]:
# LGB回归模型
# 谷歌框架
clf = lgb.LGBMRegressor(
    learning_rate=0.1,
    max_depth=-1,
    n_estimators=5000,
    random_state=2019,
    objective='regression',
    boosting_type='gbdt'
)
clf.fit(X=train_data,y=train_target,eval_metric='MSE',verbose=50)
score = mean_squared_error(test_target,clf.predict(test_data))
print("lgb:",score)